In [6]:
import pandas as pd
import math
import numpy as np

In [7]:
path = './data/data_interpolate_time_series.xlsx'
df = pd.read_excel(path)
df.shape

(3583, 20)

In [20]:
def zero_feature(s):
    length = len(s.split(' '))
    if length < 10:
        return ' '.join(['0','0','0','0','0','0','0','0','0','0'])
    else:
        return s

# 将字符串序列变为数字序列
def toNumberList(s):
    return [float(c) for c in s.split(' ')] 

# 返回一个数字组成的序列
def minus_to_zero(s):
    arr = toNumberList(s)
    ans = []
    for num in arr:
        if num < 0:
            ans.append(0)
        else:
            ans.append(num)
    return ans

# 变换时间序列(以字符串的形式输入)，是异常值标记为1，不是异常值标记为0
def trans_arr(s,col_name):
    arr = minus_to_zero(s)
    ans = []
    if col_name == 'NBPSYS' or col_name == 'NBPSYS_inter':
        for val in arr:
            if val >= 140 or val <= 90:
                ans.append(1)
            else:
                ans.append(0)
    if col_name == 'NBPDIA' or col_name == 'NBPDIA_inter':
        for val in arr:
            if val >= 90 or val <= 60:
                ans.append(1)
            else:
                ans.append(0)
    if col_name == 'PR' or col_name == 'PR_inter':
        for val in arr:
            if val >= 100 or val <= 60:
                ans.append(1)
            else:
                ans.append(0)
    if col_name == 'SPO2' or col_name == 'SPO2_inter':
        for val in arr:
            if  val <= 95:
                ans.append(1)
            else:
                ans.append(0)
    return ans
        

def feature_select(s,col_name):
    arr = trans_arr(s,col_name)
    length = len(arr)
    ans = []
    if length <= 10:
        return zero_feature(s)
    step = math.floor(length / 10)
    
    cnt = 1
    for idx in range(0,length,step):
        if cnt == 10:
            step_arr = arr[idx:length]
        else:
            step_arr = arr[idx:idx+step]
        step_val = sum(step_arr)
        ans.append(str(step_val))
        if cnt == 10:
            break
        cnt += 1
    return ' '.join(ans)

In [21]:
col_name = 'NBPSYS'
s = df.loc[4,col_name]
s_inter = df.loc[4,'{}_inter'.format(col_name)]
print('---------')
print(s)
print('---------')
print(s_inter)
print('---------')
print(feature_select(s,col_name))
print('---------')
print(feature_select(s_inter,col_name))

---------
138 -102 -102 -102 -102 -102 -102 -102 -102 -102 137 -102 -102 -102 -102 -102 -102 -102 -102 -102 132 -102 -102 -102 -102 -102 -102 -102 -102 -102 117 -102 -102 -102 -102 -102 -102 -102 -102 -102 86 -102 -102 -102 -102 -102 -102 -102 -102 -102 78 -102 -102 -102 -102 -102 -102 -102 -102 -102 76 -102 -102 -102 -102 -102 -102 -102 -102 -102 74 -102 -102 -102 -102 -102 -102 -102 -102 -102 80 -102 -102 -102 -102 -102 -102 -102 -102 -102 86 -102 -102 -102 -102 -102 -102 -102 -102 -102 103 -102 -102 -102 -102 -102 -102 -102 -102 -102 98 -102 -102 -102 -102 -102 -102 -102 -102 -102 110 110 -102 -102 -102 110 103 103 103 103 103 103 99 99 99 99 99 91 91 91 91 91 89 89 89 89 89 89 88 88 88 88 88 88 89 89 89 89 89 89 85 85 85 85 85 85 85 141 141 141 128 141 125 125 125 125 125 125 120 120 120 120 120 120 123 123 123 123 123 123 119 119 119 119 119 119 116 116 116 116 116 116 116 116 116 116 116 116 118 118 118 118 118 118 114 114 114 114 114 114 114 114 114 114 114 114 116 116 116 116 1

In [22]:
for col in ['NBPDIA','NBPSYS','PR','SPO2','NBPDIA_inter','NBPSYS_inter','PR_inter','SPO2_inter']:
    df[col+'_feature'] = df[col].apply(feature_select,args=(col,))

In [23]:
df.head(3)

,手术ID,姓名,性别,年龄,体重,身高,病区,实施手术,实施手术_分词,术前诊断,...,SPO2_inter,PR_inter,NBPDIA_feature,NBPSYS_feature,PR_feature,SPO2_feature,NBPDIA_inter_feature,NBPSYS_inter_feature,PR_inter_feature,SPO2_inter_feature
0,408485,邓煊雅,女,36,66.0,162.0,妇产科病区,二次剖宫产术,二次|剖宫产术,1.妊娠38+2周孕6产1晚孕、 2.瘢痕子宫、 3.右侧附件囊肿；,...,99.0 99.0 99.0 99.0 99.0 99.0 99.0 99.0 99.0 9...,87.0 88.5 88.5 88.5 88.5 88.5 88.5 88.5 88.5 9...,5 0 0 0 0 0 0 0 0 0,4 1 0 0 0 0 0 0 0 0,4 4 0 0 0 0 1 4 4 7,5 3 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 1 4 4 7,0 0 0 0 0 0 0 0 0 0
1,408486,刘书,女,51,58.0,160.0,心血管病区,房颤消融改良迷宫术+人工二尖瓣置换术,房颤|消融|改良|迷宫术|人工|二尖瓣|置换术,风心病,...,98.0 98.0 98.0 98.0 98.0 98.0 98.0 98.0 98.0 9...,165.0 159.0 159.0 159.0 159.0 159.0 159.0 159....,3 4 3 4 4 3 4 3 4 5,3 4 3 4 4 3 4 3 4 5,4 4 4 4 4 4 4 4 4 5,3 4 3 4 4 3 4 3 4 4,0 0 0 0 0 0 0 0 0 1,0 0 0 0 0 0 0 0 0 1,4 4 4 4 4 4 4 4 4 5,0 0 0 0 0 0 0 0 0 0
2,408487,杨仁秀,女,66,47.0,144.5,心血管病区,再次二尖瓣置换术,二尖瓣|置换术,心脏瓣膜置换术后 瓣膜功能障碍,...,98.0 99.0 99.0 99.0 100.0 100.0 100.0 100.0 10...,76.0 79.0 78.0 79.0 79.0 80.0 81.0 82.0 80.5 8...,0 0 2 0 0 0 0 0 0 0,0 0 2 0 0 0 0 0 0 0,0 2 0 0 0 0 0 0 1 4,0 2 0 0 0 0 0 0 0 2,0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 0,0 0 0 0 0 0 0 0 0 1,0 0 0 0 0 0 0 0 0 1


In [24]:
df.to_excel('./data/data_time_series_feature_select_pro.xlsx',index=None)